For quick and automated analysis, all designs' names are organized in a design_list.txt file. Simulation files for each design need to be organized in respectively named directories, subdivided into "apo" and "holo" directories.

### Necessary modules

In [ ]:
# Python libraries
import numpy as np
%pylab inline
from glob import glob
import re
import string
from scipy.stats import norm
import math

## pyEmma
from __future__ import print_function
import pyemma
print(pyemma.version)
coor=pyemma.coordinates
import pyemma.plots as mplt

# MDtraj
import mdtraj as md

# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pylab import *
from matplotlib import colors, ticker, cm
import matplotlib.mlab as mlab

# MDAnalysis
import MDAnalysis
from MDAnalysis import Universe
from MDAnalysis.analysis.density import density_from_Universe
from MDAnalysis.analysis.align import AlignTraj
from MDAnalysis.analysis.waterdynamics import SurvivalProbability as SP

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL) 

## Define path variables

In [1]:
designs_file = '/path/to/design_list.txt' # file containing all design names
traj_path = '/path/to/root/directory/containing/trajectory/files/'  ## directory to where trajectory files are located
outdir = '/path/to/results/root/directory/' # directory where analysis results will be saved to

## Number of Clusters (NOC)

Extract atom coordinates from trajectories

In [ ]:
# Apo    
designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Load trajectory
    topfile = traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'  ## prmtop file for apo simulations
    traj_list = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulations*.nc')
    traj_list.sort()# Creating a list with all trajectory files in ascending order
    
    feat = coor.featurizer(topfile)
    indices = feat.select_Ca() ## get indices of Calphas

    # Getting Ca-coordinates (coordinates are saved in x y z format)
    feat.add_selection(indices)
    inp = coor.source(traj_list, feat)
    c = inp.get_output()
    
    # Saving data
    save_object(c, outdir+'clustering/' + str(name) + '_apo_Calphas_coordinates_combinedRuns.dat')   ##

In [ ]:
# Holo    
designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Load trajectory
    topfile = traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'  ## prmtop file for holo simulations
    traj_list = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulations*.nc')
    traj_list.sort()# Creating a list with all trajectory files in ascending order
    
    feat = coor.featurizer(topfile)
    indices = feat.select_Ca() ## get indices of Calphas

    # Getting Ca-coordinates (coordinates are saved in x y z format)
    feat.add_selection(indices)
    inp = coor.source(traj_list, feat)
    c = inp.get_output()
    
    # Saving data
    save_object(c, outdir+'clustering/' + str(name) + '_holo_Calphas_coordinates_combinedRuns.dat')   ##

Perform RMSD clustering:

In [ ]:
## APO

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)

    ## Load distance file
    d = pickle.load(open(outdir+'clustering/' + str(name) + '_apo_Calphas_coordinates_combinedRuns.dat', 'rb'))
    
    #Clustering - change parameters if desired
    cl = coor.cluster_regspace(data = d, dmin = 0.15, stride=1, metric = 'minRMSD')
    print(cl)

In [ ]:
## HOLO

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)

    ## Load distance file
    d = pickle.load(open(outdir+'clustering/' + str(name) + '_holo_Calphas_coordinates_combinedRuns.dat', 'rb'))
    
    #Clustering - change parameters if desired
    cl = coor.cluster_regspace(data = d, dmin = 0.15, stride=1, metric = 'minRMSD')
    print(cl)

## Protein RMSF

Creating design-specific RMSF files from the default rmsf_analysis.in file provided with this notebook

Apo:

In [ ]:
## Creating cpptraj input to do RMSF analysis

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    n = 109 # number of residues in the protein

    for r in range (1,4):  ## Perform analysis for all 3 runs, one at a time
        ## cpptraj wrap input     
        with open(outdir+'RMSF/' + str(name) + '_RMSF.r0' + str(r) + '.in', 'w') as inp:
            infile = open(outdir+'RMSF/rmsf_analysis.in', 'r')
            i = 1
            for line in infile:
                if i == 1:
                    inp.write('trajin '+traj_path+str(name)+ '/apo/' + str(name) +'_apo_simulation.r0'+ str(r) + '.nc\n')
                    i = i + 1
                elif i == 4:
                    inp.write('atomicfluct out rmsf_CAfitCA_'+str(name) + '_apo.r0' + str(r) +'.dat   @CA byres\n')
                    i = i + 1
                elif i == 5:
                    inp.write('atomicfluct out rmsf_RESfitCA_'+str(name) + '_apo.r0' + str(r) +'.dat :1-'+str(n) + ' byres\n')
                    i = i + 1
                else:
                    inp.write(line)
                    i = i + 1

In [ ]:
%%bash
cd /path/to/design_list/file
while IFS= read -r file
        do
            cd /path/to/results/root/directory/RMSF/
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/apo/"$file"_apo.prmtop "$file"_RMSF.r01.in > RMSF.r01.log
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/apo/"$file"_apo.prmtop "$file"_RMSF.r02.in > RMSF.r02.log
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/apo/"$file"_apo.prmtop "$file"_RMSF.r03.in > RMSF.r03.log
        done < "design_list.txt"

Holo

In [ ]:
## Creating cpptraj input to do RMSF analysis

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    n = 109 # number of residues in the protein

    for r in range (1,4):  ## Perform analysis for all 3 runs, one at a time
        ## cpptraj wrap input     
        with open(outdir+'RMSF/' + str(name) + '_RMSF.r0' + str(r) + '.in', 'w') as inp:
            infile = open(outdir+'RMSF/rmsf_analysis.in', 'r')
            i = 1
            for line in infile:
                if i == 1:
                    inp.write('trajin '+traj_path+str(name)+ '/holo/' + str(name) +'_holo_simulation.r0'+ str(r) + '.nc\n')
                    i = i + 1
                elif i == 4:
                    inp.write('atomicfluct out rmsf_CAfitCA_'+str(name) + '_holo.r0' + str(r) +'.dat   @CA byres\n')
                    i = i + 1
                elif i == 5:
                    inp.write('atomicfluct out rmsf_RESfitCA_'+str(name) + '_holo.r0' + str(r) +'.dat :1-'+str(n) + ' byres\n')
                    i = i + 1
                else:
                    inp.write(line)
                    i = i + 1

In [ ]:
%%bash
cd /path/to/design_list/file
while IFS= read -r file
        do
            cd /path/to/results/root/directory/RMSF/
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/holo/"$file"_holo.prmtop "$file"_RMSF.r01.in > RMSF.r01.log
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/holo/"$file"_holo.prmtop "$file"_RMSF.r02.in > RMSF.r02.log
            cpptraj /path/to/root/directory/containing/trajectory/files/"$file"/holo/"$file"_holo.prmtop "$file"_RMSF.r03.in > RMSF.r03.log
        done < "design_list.txt"

## SASA

Saving per-residue SASA

In [ ]:
## APO

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    for r in range(1,4): ## Perform analysis for all 3 runs, one at a time
    
        ## Load trajectory
        prmtop_path =  traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'  ## prmtop file for apo simulations
        traj_file = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulations.r0'+str(r)+'.nc')
        new_traj = md.load(traj_file, top=prmtop_path, stride=100)
        
        # calculate SASA
        topology = new_traj.topology
        sasa = md.shrake_rupley(new_traj.atom_slice(topology.select('protein')),  probe_radius=0.14, n_sphere_points=960, mode='residue')
        ## save as object
        save_object(sasa, outdir+'SASA/' + str(name) + '_apo_sasa_stride100_r0'+str(r)+'.dat')   ##

In [ ]:
## HOLO

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    for r in range(1,4): ## Perform analysis for all 3 runs, one at a time
    
        ## Load trajectory
        prmtop_path =  traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'  ## prmtop file for holo simulations
        traj_file = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulations.r0'+str(r)+'.nc')
        new_traj = md.load(traj_file, top=prmtop_path, stride=100)
        
        # calculate SASA
        topology = new_traj.topology
        sasa = md.shrake_rupley(new_traj.atom_slice(topology.select('protein')),  probe_radius=0.14, n_sphere_points=960, mode='residue')
        ## save as object
        save_object(sasa, outdir+'SASA/' + str(name) + '_holo_sasa_stride100_r0'+str(r)+'.dat')   ##

Calculating hydrophobic SASA

In [ ]:
designs = open(designs_file, 'r')

apo = []
stdev_apo = []
holo = []
stdev_holo = []

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Load sample structure to get residue ids
    prmtop_path =  traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'
    traj_file = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulations.r01.nc')
    new_traj = md.load(traj_file, top=prmtop_path, stride=100)

    topology = new_traj.topology
    backbone = topology.select('name CA')
    # list of hydrophobic residues
    backbone_hydrophobic = topology.select('name CA and resname ALA or name CA and resname ILE or name CA and resname LEU or name CA and resname PHE or name CA and resname VAL or name CA and resname PRO or name CA and resname GLY or name CA and resname MET or name CA and resname TRP')

    # Creat a list with the positions of hydrophobic residues in the SASA output
    hydrophobic_res_column = []
    for y in range (0, len(backbone_hydrophobic)):
        for z in range (0, len(backbone)):
            if backbone_hydrophobic[y] == backbone[z]:
                hydrophobic_res_column.append(z)
                
    apo_runs = []
    holo_runs = []
    
    for r in range(1,4):
        sasa = pickle.load(open(outdir+'SASA/'+str(name) + '_apo_sasa_stride100_r0'+str(r)+'.dat', 'rb'))
        hydrophobic_sasa = sasa[:50, hydrophobic_res_column] ## Get only columns corresponding to hydrophobic residues, for first 500 ns
        apo_runs.append((hydrophobic_sasa.sum(axis=1))/len(hydrophobic_res_column))
        
        sasa = pickle.load(open(outdir+'SASA/'+ str(name) + '_holo_sasa_stride100_r0'+str(r)+'.dat', 'rb'))
        hydrophobic_sasa = sasa[:50, hydrophobic_res_column] ## Get only columns corresponding to hydrophobic residues, for first 500 ns
        holo_runs.append((hydrophobic_sasa.sum(axis=1))/len(hydrophobic_res_column))
        
    apo.append(np.average(np.concatenate(apo_runs,axis=0)))
    stdev_apo.append(np.std(np.concatenate(apo_runs,axis=0)))
    holo.append(np.average(np.concatenate(holo_runs,axis=0)))
    stdev_holo.append(np.std(np.concatenate(holo_runs,axis=0)))

## Ligand RMSD

A pdb file with the initial complex structure (with waters) used to initiate the simulations is needed as the reference structure to calculate RMSD, here called $designname_holo.pdb

Here we write design-specific RMSD files from the default ligand_RMSD.in file provided with this notebook

In [ ]:
## Creating cpptraj input to do RMSD analysis

designs = open(designs_file, 'r')

for line in designs:
    name = line.partition("\n")[0]
    n = 109 # number of residues in the protein
    
    for r in range (1,4):
        ## creating cpptraj RMSD input     
        with open(outdir+'RMSD/' + str(name) + '_ligandRMSD.r0' + str(r) + '.in', 'w') as inp:
            infile = open(outdir+'RMSD/ligand_RMSD.in', 'r')
            i = 1
            for line in infile:
                if i == 1:
                    inp.write('parm '+ outdir + 'RMSD/' + str(name) + '_holo.pdb\n')
                    i = i + 1
                elif i == 3:
                    inp.write('parm '+ traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop\n')
                    i = i + 1
                elif i == 4:
                    inp.write('trajin '+traj_path+str(name)+'/holo/'+str(name) +'_holo_simulation.r0' + str(r) + '.nc parmindex 1\n')
                    i = i + 1
                elif i == 6:
                    inp.write('reference '+ outdir + 'RMSD/'+ str(name) + '_holo.pdb parmindex 0\n')
                    i = i + 1
                elif i == 8:
                    inp.write('rms reference :1-' + str(n) +'@CA\n')
                    i = i + 1
                elif i == 9:
                    inp.write('rmsd :' + str(int(n)+1)+ ' reference nofit out ' + str(name) + '_ligandRMSD.r0' +str(r) +'.dat\n')
                    i = i + 1
                else:
                    inp.write(line)
                    i = i + 1

In [ ]:
%%bash
cd /path/to/design_list/file
while IFS= read -r file
        do
            cd /path/to/results/root/directory/RMSD/
            cpptraj "$file"_holo.pdb "$file"_ligandRMSD.r01.in > RMSD.r01.log
            cpptraj "$file"_holo.pdb "$file"_ligandRMSD.r02.in > RMSD.r02.log
            cpptraj "$file"_holo.pdb "$file"_ligandRMSD.r03.in > RMSD.r03.log
        done < "design_list.txt"

## Protein-ligand H bonds count

In [ ]:
## Getting ids of atoms that interact with the ligand for at least 1% of the time

designs = open(designs_file, 'r')

pairs = []

for line in designs:
    name = line.partition("\n")[0]
    n = 109 # number of residues in the protein
    
    topfile =  traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'  ## prmtop file for apo simulations
    traj_list = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulations.r0*.nc')
    traj_list.sort()# Creating a list with all trajectory files in ascending order
    
    new_traj = md.load(traj_list[0], top=topfile, stride=1)
    
    for r in range (1,len(traj_list)):
        temp_traj= md.load(traj_list[r], top=topfile, stride=1)
        new_traj = new_traj + temp_traj

    hbonds = md.baker_hubbard(new_traj, periodic=False, freq=0.01)
    label = lambda hbond : '%s -- %s' % (new_traj.topology.atom(hbond[0]), new_traj.topology.atom(hbond[2]))
    design_pairs = []
    for hbond in hbonds:
        if str(new_traj.topology.atom(hbond[0]).residue) == 'MOL'+str(n) or str(new_traj.topology.atom(hbond[2]).residue) == 'MOL'+str(n):
            print(label(hbond))
            design_pairs.append(hbond)
    pairs.append(design_pairs)

In [ ]:
## Calculating angle and distances between the pairs

designs = open(designs_file, 'r')

d = 0 ## keeping count of which design we're in
average_hbond_per_frame = []

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    hbonds =  np.array(pairs[d]) ## getting the hbonds ids of the design

    topfile = traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'  
    
    hbond_per_frame = []  ## to contain count for each frame of the combined runs
    
    for r in range(1,4):
        traj_list = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulations.r0'+str(r)+'.nc')
    
        new_traj = md.load(traj_list[0], top=topfile, stride=1)
                 
        ## Calculate X-H --- Y distance
        dist  = (md.compute_distances(new_traj, hbonds[:, [1,2]], periodic=False))*10  ## Calculating X-H --- Y distance (that is, distance between H and Y), in Angstroms
    
        ## Calculate X-H --- Y angle
        a  = (md.compute_angles(new_traj, hbonds))*57.2958 ## Angle in degrees
        
        # save distances and angles as objects
        save_object(dist, outdir +'Hbonds/'+ str(name) + '_all_protein_ligand_Hbonds_distances.r0'+str(r)+'.dat')
        save_object(a, outdir +'Hbonds/' + str(name) + '_all_protein_ligand_Hbonds_angles.r0'+str(r)+'.dat')
        
    d = d + 1

Counting H bonds within angle and distance definitions:

In [ ]:
designs = open(designs_list, 'r')

average_hbond_per_frame = []
stdev = []

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    hbond_per_frame = []
    
    for r in range(1,4):
        # open distances and angles as objects
        a = pickle.load(open(outdir +'Hbonds/'+ str(name) +'_all_protein_ligand_Hbonds_angles.r0'+str(r)+'.dat', 'rb'))
        dist = pickle.load(open(outdir +'Hbonds/'+ str(name) +'_all_protein_ligand_Hbonds_distances.r0'+str(r)+'.dat', 'rb'))
        
        ## Counting number of established H bonds per frame
        for frame in range(0,len(dist)): ## For each frame
            count = 0 # resetting cont
            for pair in range(0,len(dist[0])): ## For each pair
                if dist[frame][pair] < 3.2 and a[frame][pair] > 90: ## Using cutoff of 3.2 A and 90 degrees
                    count = count + 1
            hbond_per_frame.append(count)
            
    average_hbond_per_frame.append(np.average(hbond_per_frame))    
    stdev.append(np.average(hbond_per_frame))

## Pocket volume

Saving trajectories with a stride of 100, aligned to reference designs used to define inclusion spheres for POVME calculation:

In [ ]:
## APO

designs = open(designs_list, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ref = indir + str(name) +'_aligned.pdb' ## Reference pdb to which trajectories are aligned to  
    ref_traj = md.load(ref)
    
    for r in range (1,4):
        traj_file = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulation.r0'+ str(r) +  '.nc')
        prmtop_path = traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'
        
        new_traj = md.load(traj_file, top=prmtop_path, stride=100)
        only_protein = new_traj.atom_slice(atom_indices=new_traj.topology.select('protein'))
        aligned_traj = only_protein.superpose(ref_traj, frame=0, atom_indices=only_protein.topology.select('protein and backbone'), ref_atom_indices=ref_traj.topology.select('protein and backbone') )
        aligned_traj.save(outdir +'POVME/' + str(name) + '_apo_stride100_r0'+ str(r) +  '_aligned.pdb')

In [ ]:
## APO

designs = open(designs_list, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ref = indir + str(name) +'_aligned.pdb' ## Reference pdb to which trajectories are aligned to  
    ref_traj = md.load(ref)
    
    for r in range (1,4):
        traj_file = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulation.r0'+ str(r) +  '.nc')
        prmtop_path = traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'
        
        new_traj = md.load(traj_file, top=prmtop_path, stride=100)
        only_protein = new_traj.atom_slice(atom_indices=new_traj.topology.select('protein'))
        aligned_traj = only_protein.superpose(ref_traj, frame=0, atom_indices=only_protein.topology.select('protein and backbone'), ref_atom_indices=ref_traj.topology.select('protein and backbone') )
        aligned_traj.save(outdir +'POVME/' + str(name) + '_holo_stride100_r0'+ str(r) +  '_aligned.pdb')

Create input files for POVME, from default POVME_input.ini, in which the inclusion and seed spheres have been defined

In [ ]:
# Apo
designs = open(designs_list, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    for r in range(1,4):
        with open(outdir + 'POVME/'+ str(name) + '_r0' + str(r) + '_apo_POVME.ini', 'w') as inp:
    
            infile = open(outdir + 'POVME/'+ 'POVME_input.ini', 'r')
            i = 1
            for line in infile:
                if i == 14:
                    inp.write('OutputFilenamePrefix  ' + str(name) + '_apo_stride100_r0' + str(r) +'_POVME2.0/\n')
                    i = i + 1
                elif i == 15:
                    inp.write('PDBFileName ' + str(name) +'_apo_stride100_r0' + str(r) + '_aligned.pdb\n')
                    i = i + 1
                else:
                    inp.write(line)
                    i = i + 1

In [ ]:
# Apo
designs = open(designs_list, 'r')

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    for r in range(1,4):
        with open(outdir + 'POVME/'+ str(name) + '_r0' + str(r) + '_holo_POVME.ini', 'w') as inp:
    
            infile = open(outdir + 'POVME/'+ 'POVME_input.ini', 'r')
            i = 1
            for line in infile:
                if i == 14:
                    inp.write('OutputFilenamePrefix  ' + str(name) + '_holo_stride100_r0' + str(r) +'_POVME2.0/\n')
                    i = i + 1
                elif i == 15:
                    inp.write('PDBFileName ' + str(name) +'_holo_stride100_r0' + str(r) + '_aligned.pdb\n')
                    i = i + 1
                else:
                    inp.write(line)
                    i = i + 1

In [ ]:
## Creating a list with all input files in order to run in bash

designs = open(designs_file, 'r')

with open(outdir + 'POVME/'+ 'POVME_input_files.txt', 'w') as inp:
    for line in designs:
        name = line.partition("\n")[0]
        print(name)
    
        for r in range(1,4):
            inp.write(str(name) + '_r0' + str(r) +'_apo_POVME.ini\n')
            inp.write(str(name) + '_r0' + str(r) +'_holo_POVME.ini\n')

Run in terminal using the provided run.sh file with ./run.sh. Cavity volume per frame is saved in volumes.tabbed.txt in the respective output directories

## Side chain dihedral angles

This hasn't been automated, so the desired residues for which to calculate dihedral angles need to be specified as exemplified below

In [ ]:
# Select residues
hbi10_residues = [10, 22, 50, 70, 88, 90, 98, 100]
hbi10_residues_names = ['TRP', 'THR', 'PHE', 'TYR', 'PHE', 'PHE', 'THR', 'PHE']
hbi11_residues = [10, 16, 22, 28, 70, 88, 90, 94]
hbi11_residues_names = ['TRP', 'ASN', 'SER', 'PHE', 'TYR', 'PHE', 'PHE', 'THR']

residues = [hbi10_residues,hbi11_residues]
names = [hbi10_residues_names,hbi11_residues_names]

In [ ]:
designs = open(designs_list, 'r')

d = 0 ## keeping count of which design we're in
average_hbond_per_frame = []

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    ids =  np.array(residues[d]) ## getting the residue ids of the design 
    
    topfile = traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'  
    traj_list = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulations.r0*.nc')
    traj_list.sort()# Creating a list with all trajectory files in ascending order
    
    new_traj = md.load(traj_list[0], top=topfile, stride=1)  
    for r in range (1,len(traj_list)):
        temp_traj= md.load(traj_list[r], top=topfile, stride=1)
        new_traj = new_traj + temp_traj
    
    chi1_indices, chi1_angles = md.compute_chi1(new_traj)  
    residues_chi1 = []
    
    for i in range(0, len(ids)): 

        alpha_c = new_traj.topology.select('resid ' + str(ids[i])+ ' and name CA')  
        
        dihedral_index = np.argwhere(chi1_indices == alpha_c)[0][0]
        for f in range (len(chi1_angles[:, dihedral_index])):
            chi1_angles[f,dihedral_index]=math.degrees(chi1_angles[f,dihedral_index])
        residues_chi1.append(chi1_angles[:,dihedral_index])
        time=range(len(chi1_angles[:,0]))
        
    ## Save object
    save_object(residues_chi1, outdir +'dihedral_angle/' + str(name) + '_selected_atoms_dihedrals_apo.dat')

    # Calculating % of frames with designed rotamer orientation
    output = open(outdir +'dihedral_angle/'+str(name)+'_apo_%similarityDesignRotamer', 'w') ##
    cutoff = 30.00  # angle cutoff range
    ref_residue_id = 0

    for dihedral in range (len(residues_chi1)): ##  For each dihedral
        correct_rotamer = 0
        for frame in range (len(residues_chi1[0])): ## For each frame
            if ref_residues_chi1[ref_residue_id][0] <= -150: # to account for the -180 +180 angles
                if ref_residues_chi1[ref_residue_id][0] + cutoff >= residues_chi1[dihedral][frame] or residues_chi1[dihedral][frame] >= 150: ## < -150 or > 150
                    correct_rotamer = correct_rotamer + 1
            elif ref_residues_chi1[ref_residue_id][0] >= 150: # to account for the -180 +180 angles
                if ref_residues_chi1[ref_residue_id][0] - cutoff <= residues_chi1[dihedral][frame] or residues_chi1[dihedral][frame] <= -150: ## < -150 or > 150
                    correct_rotamer = correct_rotamer + 1
            else:
                if ref_residues_chi1[ref_residue_id][0] - cutoff <= residues_chi1[dihedral][frame] <= ref_residues_chi1[ref_residue_id][0] + cutoff: ##
                    correct_rotamer = correct_rotamer + 1
        average_correct_rotamer = (correct_rotamer/float(len(residues_chi1[0]))) *100 ##
        print(str(name) + ' apo '+ names[d][dihedral] + str(residues[d][dihedral]+1)+ ' Chi 1 % frames predicted rotamer = ' + str(average_correct_rotamer))
        output.write(str(name) + ' holo '+ names[d][dihedral] + str(residues[d][dihedral]+1)+ ' Chi 1 % frames predicted rotamer = ' + str(average_correct_rotamer) + '\n')
        
        ref_residue_id =ref_residue_id + 1
    
    output.close()
        
    d = d + 1

## Water count

In here I use MDAnalysis' volume definition defined around a point in space as the center of a sphere. This is good because it's design independent, and I'm selecting the position of atom C4 in the ligand of the reference design as the center coordinate, since this is really at the center of the pocket. Simulations were aligned to the reference structure in order for this to work.

In [ ]:
## APO

designs = open(designs_file, 'r')

c4 = re.compile(r'C4  HBI X')  ## C4 is the selected central atom to define the inclusion sphere for water calculation

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Find coordinates of C4 atom
    with open(outdir+ 'water_count/'+str(name) +'_holo.pdb', 'r') as ref:
        for line in ref:
            if c4.search(line):
                columns = line.split()
                x = float(columns[6])
                y = float(columns[7])
                z = float(columns[8])
    print(x,y,z)
    
    runs = []

    topology = traj_path + str(name) + '/apo/' + str(name) +'_apo.prmtop'
    
    ## calculate number of waters for each run
    for r in range(1,4):
        water_count = []
        traj = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulation.r0'+str(r)+'.nc')
        u = Universe(topology,traj)
        group = u.select_atoms("resname WAT and name O and point " +str(x)+ " " +str(y)+" "+str(z)+" 7",updating=True) ## Coordinates of ligand's atom C4
        frame = 0
        while frame < 4999: # number of frames in trajectory
            water_count.append(len(group.positions)) ## positions gives the coordinates for each selected atom. 
                                                    ## Calculating the length of this list was the only way I found to give the number of water molecules
            frame = frame + 1
            u.trajectory.next() # going to next frame in the trajectory
        runs.append(water_count)
    
    ## save object contaning the water count for each run, for each design
    save_object(runs, outdir + 'water_count/'+ str(name) +'_apo_water_count_per_run.dat')   ##

In [ ]:
## HOLO

designs = open(designs_list, 'r')

c4 = re.compile(r'C4  HBI X')  ## C4 is the selected central atom to define the inclusion sphere for water calculation

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Find coordinates of C4 atom
    with open(outdir+ 'water_count/'+str(name) +'_holo.pdb', 'r') as ref:
        for line in ref:
            if c4.search(line):
                columns = line.split()
                x = float(columns[6])
                y = float(columns[7])
                z = float(columns[8])
    print(x,y,z)
    
    runs = []

    topology = traj_path + str(name) + '/holo/' + str(name) +'_holo.prmtop'
    
    ## calculate number of waters for each run
    for r in range(1,4):
        water_count = []
        traj = glob(traj_path + str(name) + '/holo/' + str(name) + '_holo_simulation.r0'+str(r)+'.nc')
        u = Universe(topology,traj)
        group = u.select_atoms("resname WAT and name O and point " +str(x)+ " " +str(y)+" "+str(z)+" 7",updating=True) ## Coordinates of ligand's atom C4
        frame = 0
        while frame < 4999: # number of frames in trajectory
            water_count.append(len(group.positions)) ## positions gives the coordinates for each selected atom. 
                                                    ## Calculating the length of this list was the only way I found to give the number of water molecules
            frame = frame + 1
            u.trajectory.next() # going to next frame in the trajectory
        runs.append(water_count)
    
    ## save object contaning the water count for each run, for each design
    save_object(runs, outdir + 'water_count/'+ str(name) +'_holo_water_count_per_run.dat')   ##

## Water survival probability

Calculating for apo simulations only:

In [ ]:
designs = open(designs_list, 'r')

c4 = re.compile(r'C4  HBI B')  ## C4 is the selected central atom to define the inclusion sphere for water calculation

for line in designs:
    name = line.partition("\n")[0]
    print(name)
    
    ## Find coordinates of C4 atom
    with open(outdir+ 'water_count/'+str(name) +'_holo.pdb', 'r') as ref:
        for line in ref:
            if c4.search(line):
                columns = line.split()
                x = float(columns[6])
                y = float(columns[7])
                z = float(columns[8])
    print(x,y,z)
    
    topology = indir + 'water_survival/' + str(name) + '/apo/' + str(name) +'_apo.prmtop'
    
    ## calculate number of waters for each run
    for r in range(1,4):
        traj = glob(traj_path + str(name) + '/apo/' + str(name) + '_apo_simulations.r0'+str(r)+'.nc')
        u = Universe(topology,traj)
        selection = "resname WAT and name O and point " +str(x)+ " " +str(y)+" "+str(z)+" 7" ## Coordinates of ligand's atom C4
        SP_analysis = SP(u, selection, 4000, 5000, 100)
        SP_analysis.run()

        ## save object 
        save_object(SP_analysis.timeseries, outdir + 'water_survival/' str(name) +'_apo_water_SP.r0' +str(r) +'.dat')   ##

## Unsupervised learning model

Organize above calculated features in arrays:

In [ ]:
NOC_apo = [...]
NOC_holo = [...]
SASA_apo = [...]
SASA_holo = [...]

In [ ]:
vol_apo = [...]
vol_holo = [...]
below_30_frame_count_apo = [...]
vol_change_apo = [...]
vol_change_holo = [...]

In [ ]:
water_count_apo =[...]
water_count_holo =[...]
ligand_rmsd = [...]
protein_ligand_hbonds = [...]

Creating array with all features:

In [ ]:
data = []

data.append(NOC_apo)
data.append(NOC_holo)
data.append(SASA_apo)
data.append(SASA_holo)
data.append(vol_apo)
data.append(vol_holo)
data.append(below_30_frame_count_apo)
data.append(vol_change_apo)
data.append(vol_change_holo)
data.append(water_count_apo)
data.append(water_count_holo)
data.append(ligand_rmsd)
data.append(protein_ligand_hbonds)

In [ ]:
feature_names = ['Apo NOC','Holo NOC','Apo SASA','Holo SASA','Apo cavity volume','Holo cavity volume','Apo volume frame count <cutoff','Apo volume change','Holo volume change','Apo cavity water count','Holo cavity water count','Ligand RMSD','Protein-ligand H bonds count']

In [ ]:
features = np.transpose(np.asarray(data))
print(features)

In [ ]:
len(features)

PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## This scales each feature in the array (that is, each column in the features array) to range from 0 to 1 according
## to its minimum and maximun values
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
print(scaled_features)

In [ ]:
# PCA with three components
pca3 = PCA(n_components=3)
reduced_cartesian_scaled = pca3.fit_transform(scaled_features)
print(reduced_cartesian_scaled.shape)
print(pca3.explained_variance_ratio_)  

In [ ]:
# Plot 3D scatter

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

for i in range(0,len(design_names)):
    if design_names[i] in good_binders:
        ax.scatter(reduced_cartesian_scaled[i, 0],  reduced_cartesian_scaled[i,1], reduced_cartesian_scaled[i,2], color = 'darkturquoise', s=50)
    elif design_names[i] in unstable:
        ax.scatter(reduced_cartesian_scaled[i, 0],  reduced_cartesian_scaled[i,1], reduced_cartesian_scaled[i,2], color = 'black', s=50)
    else:
        ax.scatter(reduced_cartesian_scaled[i, 0],  reduced_cartesian_scaled[i,1], reduced_cartesian_scaled[i,2], color = 'orangered', s=50)

axes=plt.gca()        

ax.set_xlabel('Principal Component 1',fontsize=12)
ax.set_ylabel('Principal Component 2',fontsize=12)
ax.set_zlabel('Principal Component 3',fontsize=12)

#fig.savefig('/scratch/epecoradebarros/Baker_project/Analysis_paper/298K_biggerBox/model/Bbarrel_3PCA_plot.pdf', bbox_inches = 'tight')

In [ ]:
plt.matshow(pca3.components_,cmap='viridis')
plt.yticks([0,1,2],['PC 1','PC 2','PC 3'],fontsize=12)
cbar = plt.colorbar()
cbar.ax.set_ylabel('Feature contribution',fontsize=12)
plt.xticks(range(len(feature_names)),feature_names,rotation=65,ha='left',fontsize=16)
#plt.savefig('/gpfs/amarolab/epecoradebarros/texas_scratch/Baker_project/Analysis_paper/298K_biggerBox/model/Bbarrel_3PC_feature_contriburion_largerFont.pdf', bbox_inches = 'tight')

Clustering -  3 clusters

In [ ]:
# import KMeans
from sklearn.cluster import KMeans

In [ ]:
# create kmeans object
kmeans = KMeans(n_clusters=3)
# fit kmeans object to data
kmeans.fit(reduced_cartesian_scaled)
# print location of clusters learned by kmeans object
print(kmeans.cluster_centers_)
# save new clusters for chart
y_km = kmeans.fit_predict(reduced_cartesian_scaled)

In [ ]:
## Seeing results
plt.scatter(reduced_cartesian_scaled[y_km ==0,0], reduced_cartesian_scaled[y_km == 0,1], s=100, c='red')
plt.scatter(reduced_cartesian_scaled[y_km ==1,0], reduced_cartesian_scaled[y_km == 1,1], s=100, c='black')
plt.scatter(reduced_cartesian_scaled[y_km ==2,0], reduced_cartesian_scaled[y_km == 2,1], s=100, c='blue')

## Supervised learning models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, f1_score, matthews_corrcoef, precision_score, recall_score
from sklearn.model_selection import KFold

Organize features and labels for data:

In [ ]:
X = features
y = [0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0]
## labels: 0 for non-binder, 1 for binder

Performing k-fold cross validation:

- Logistic regression classifier

In [ ]:
mcc = []
roc = []
f1s = []
acc = []
prec = []
rec = []

kf = KFold(n_splits=5, shuffle = True)  ## n_splits = 5 for 5-fold cross validation
kf.get_n_splits(X)
print(kf)  
for train_index, test_index in kf.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = np.array(y)[train_index], np.array(y)[test_index]
    print(y_train)
    
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_valid = scaler.transform(X_valid)
    
    # Fitting a logistic regression
    lrc = LogisticRegression().fit(X_train,y_train)
    
    # Validation
    probs = lrc.predict_proba(X_valid)[:,1]
    preds = lrc.predict(X_valid)
    mcc.append(matthews_corrcoef(y_valid, preds))
    roc.append(roc_auc_score(y_valid, probs))
    f1s.append(f1_score(y_valid, preds))
    acc.append(lrc.score(X_valid,y_valid))
    prec.append(precision_score(y_valid, preds))
    rec.append(recall_score(y_valid, preds))

print('mcc = '+str(np.average(mcc))+' +- '+str(np.std(mcc)))
print('f1s = '+str(np.average(f1s))+' +- '+str(np.std(f1s)))
print('accuracy = '+str(np.average(acc))+' +- '+str(np.std(acc)))
print('precision = '+str(np.average(prec))+' +- '+str(np.std(prec)))
print('recall = '+str(np.average(rec))+' +- '+str(np.std(rec)))

- k-nearest neighbors

In [ ]:
mcc = []
roc = []
f1s = []
acc = []
prec = []
rec = []

kf = KFold(n_splits=5, shuffle = True)  ## n_splits = 5 for 5-fold cross validation
kf.get_n_splits(X)
print(kf)  
for train_index, test_index in kf.split(X):   
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = np.array(y)[train_index], np.array(y)[test_index]
    print(y_train)
    
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_valid = scaler.transform(X_valid)
    
    # Fitting a logistic regression
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train,y_train)
    
    # Validation
    probs = knn.predict_proba(X_valid)[:,1]
    preds = knn.predict(X_valid)
    mcc.append(matthews_corrcoef(y_valid, preds))
    roc.append(roc_auc_score(y_valid, probs))
    f1s.append(f1_score(y_valid, preds))
    acc.append(knn.score(X_valid,y_valid))
    prec.append(precision_score(y_valid, preds))
    rec.append(recall_score(y_valid, preds))

print('mcc = '+str(np.average(mcc))+' +- '+str(np.std(mcc)))
print('f1s = '+str(np.average(f1s))+' +- '+str(np.std(f1s)))
print('accuracy = '+str(np.average(acc))+' +- '+str(np.std(acc)))
print('precision = '+str(np.average(prec))+' +- '+str(np.std(prec)))
print('recall = '+str(np.average(rec))+' +- '+str(np.std(rec)))